# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current_Description
0,0,Sandnessjoen,NO,66.0217,12.6316,48.42,94,75,6.91,overcast clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,40.66,75,0,1.14,overcast clouds
2,2,Mahebourg,MU,-20.4081,57.7000,75.25,73,40,12.66,overcast clouds
3,3,Hermanus,ZA,-34.4187,19.2345,53.80,65,74,4.21,overcast clouds
4,4,La Rioja,ES,42.2500,-2.5000,46.18,54,46,5.88,overcast clouds
5,5,Kruisfontein,ZA,-34.0033,24.7314,56.88,76,50,8.34,overcast clouds
6,6,Upernavik,GL,72.7868,-56.1549,21.51,76,100,5.32,overcast clouds
7,7,Nacala,MZ,-14.5428,40.6728,78.37,62,2,6.17,overcast clouds
8,8,Umm Lajj,SA,25.0213,37.2685,83.61,57,0,3.24,overcast clouds
9,9,Altayskoye,RU,51.9533,85.3325,43.57,61,2,3.69,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [4]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("Please insert your minimum tempurature preference for your trip?"))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("Please insert your maximum tempurature preference for your trip?"))

Please insert your minimum tempurature preference for your trip?40
Please insert your maximum tempurature preference for your trip?56


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [5]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current_Description
0,0,Sandnessjoen,NO,66.0217,12.6316,48.42,94,75,6.91,overcast clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,40.66,75,0,1.14,overcast clouds
3,3,Hermanus,ZA,-34.4187,19.2345,53.80,65,74,4.21,overcast clouds
4,4,La Rioja,ES,42.2500,-2.5000,46.18,54,46,5.88,overcast clouds
9,9,Altayskoye,RU,51.9533,85.3325,43.57,61,2,3.69,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                191
City                   191
Country                188
Lat                    191
Lng                    191
Max Temp               191
Humidity               191
Cloudiness             191
Wind Speed             191
Current_Description    191
dtype: int64

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [7]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna(subset = ['Country'])

# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current_Description
0,0,Sandnessjoen,NO,66.0217,12.6316,48.42,94,75,6.91,overcast clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,40.66,75,0,1.14,overcast clouds
3,3,Hermanus,ZA,-34.4187,19.2345,53.80,65,74,4.21,overcast clouds
4,4,La Rioja,ES,42.2500,-2.5000,46.18,54,46,5.88,overcast clouds
9,9,Altayskoye,RU,51.9533,85.3325,43.57,61,2,3.69,overcast clouds
12,12,Bredasdorp,ZA,-34.5322,20.0403,54.63,69,19,4.83,overcast clouds
14,14,Havre-Saint-Pierre,CA,50.2334,-63.5986,47.28,81,9,16.35,overcast clouds
21,21,Mar Del Plata,AR,-38.0023,-57.5575,52.09,88,95,5.99,overcast clouds
22,22,Matagami,CA,49.7502,-77.6328,51.22,90,98,5.57,overcast clouds
25,25,Koscierzyna,PL,54.1223,17.9812,51.30,94,4,9.13,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [8]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current_Description"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current_Description
0,Sandnessjoen,NO,48.42,66.0217,12.6316,overcast clouds
1,Ushuaia,AR,40.66,-54.8000,-68.3000,overcast clouds
3,Hermanus,ZA,53.80,-34.4187,19.2345,overcast clouds
4,La Rioja,ES,46.18,42.2500,-2.5000,overcast clouds
9,Altayskoye,RU,43.57,51.9533,85.3325,overcast clouds
12,Bredasdorp,ZA,54.63,-34.5322,20.0403,overcast clouds
14,Havre-Saint-Pierre,CA,47.28,50.2334,-63.5986,overcast clouds
21,Mar Del Plata,AR,52.09,-38.0023,-57.5575,overcast clouds
22,Matagami,CA,51.22,49.7502,-77.6328,overcast clouds
25,Koscierzyna,PL,51.30,54.1223,17.9812,overcast clouds


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [9]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current_Description,Hotel Name
0,Sandnessjoen,NO,48.42,66.0217,12.6316,overcast clouds,
1,Ushuaia,AR,40.66,-54.8000,-68.3000,overcast clouds,
3,Hermanus,ZA,53.80,-34.4187,19.2345,overcast clouds,
4,La Rioja,ES,46.18,42.2500,-2.5000,overcast clouds,
9,Altayskoye,RU,43.57,51.9533,85.3325,overcast clouds,
12,Bredasdorp,ZA,54.63,-34.5322,20.0403,overcast clouds,
14,Havre-Saint-Pierre,CA,47.28,50.2334,-63.5986,overcast clouds,
21,Mar Del Plata,AR,52.09,-38.0023,-57.5575,overcast clouds,
22,Matagami,CA,51.22,49.7502,-77.6328,overcast clouds,
25,Koscierzyna,PL,51.30,54.1223,17.9812,overcast clouds,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [10]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [11]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current_Description,Hotel Name
0,Sandnessjoen,NO,48.42,66.0217,12.6316,overcast clouds,Syv Søstre Apartments
1,Ushuaia,AR,40.66,-54.8000,-68.3000,overcast clouds,Albatross Hotel
3,Hermanus,ZA,53.80,-34.4187,19.2345,overcast clouds,Misty Waves Boutique Hotel
4,La Rioja,ES,46.18,42.2500,-2.5000,overcast clouds,Casa Concejos
9,Altayskoye,RU,43.57,51.9533,85.3325,overcast clouds,ЭКО-ТРЕВЕЛ АЛТАЙ
12,Bredasdorp,ZA,54.63,-34.5322,20.0403,overcast clouds,Bredasdorp Country Manor
14,Havre-Saint-Pierre,CA,47.28,50.2334,-63.5986,overcast clouds,Auberge Niapiskau 2
21,Mar Del Plata,AR,52.09,-38.0023,-57.5575,overcast clouds,Gran Hotel Mar del Plata
22,Matagami,CA,51.22,49.7502,-77.6328,overcast clouds,Motel Le Caribou
25,Koscierzyna,PL,51.30,54.1223,17.9812,overcast clouds,Bazuny Hotel & SPA Kaszuby


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [13]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset = ['Hotel Name'])

# Display sample data
clean_hotel_df.reset_index(drop=True)

,City,Country,Max Temp,Lat,Lng,Current_Description,Hotel Name
0,Sandnessjoen,NO,48.42,66.0217,12.6316,overcast clouds,Syv Søstre Apartments
1,Ushuaia,AR,40.66,-54.8000,-68.3000,overcast clouds,Albatross Hotel
2,Hermanus,ZA,53.80,-34.4187,19.2345,overcast clouds,Misty Waves Boutique Hotel
3,La Rioja,ES,46.18,42.2500,-2.5000,overcast clouds,Casa Concejos
4,Altayskoye,RU,43.57,51.9533,85.3325,overcast clouds,ЭКО-ТРЕВЕЛ АЛТАЙ
...,...,...,...,...,...,...,...
186,Igrim,RU,44.92,63.1906,64.4162,overcast clouds,Obshchezhitiye Uchebno-Proizvodstvennogo Tsentra
187,Rochegda,RU,53.29,62.8000,43.2500,overcast clouds,
188,Senno,RU,48.25,59.5336,33.9167,overcast clouds,
189,Dolores,AR,43.41,-36.3132,-57.6792,overcast clouds,Apart Hotel - Arianos


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [14]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file,index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [15]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current_Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [16]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Syv Søstre Apartments</dd>\n<dt>City</dt><dd>Sandnessjoen</dd>\n<dt>Country</dt><dd>NO</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 48.42 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Albatross Hotel</dd>\n<dt>City</dt><dd>Ushuaia</dd>\n<dt>Country</dt><dd>AR</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 40.66 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Misty Waves Boutique Hotel</dd>\n<dt>City</dt><dd>Hermanus</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 53.8 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Casa Concejos</dd>\n<dt>City</dt><dd>La Rioja</dd>\n<dt>Country</dt><dd>ES</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 46.18 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>ЭКО-ТРЕВЕЛ АЛТАЙ</dd>\n<dt>City</dt><dd>Altayskoye</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 43.57 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Br

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [17]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,66.0217,12.6316
1,-54.8000,-68.3000
3,-34.4187,19.2345
4,42.2500,-2.5000
9,51.9533,85.3325
12,-34.5322,20.0403
14,50.2334,-63.5986
21,-38.0023,-57.5575
22,49.7502,-77.6328
25,54.1223,17.9812


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [19]:
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(50.0, 51.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))